<a href="https://colab.research.google.com/github/arishp/veltech_genai/blob/main/langchain_data_connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install langchain
# !pip install langchain-community
# !pip install langchain-huggingface
# !pip install sentence-transformers
# !pip install chromadb
# !pip install accelerate
# !pip install langchain-google-genai

In [7]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_huggingface import HuggingFaceEndpoint

# Document Loaders

In [8]:
loader = TextLoader('sample.txt')
documents = loader.load()
len(documents)

1

# Document Transformers

In [9]:
text_splitter = CharacterTextSplitter(separator=" ", chunk_size=200, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
len(texts)

152

In [10]:
texts[:2]

[Document(metadata={'source': 'sample.txt'}, page_content='Mahendra Singh Dhoni (/məˈheɪndrə ˈsɪŋ dhæˈnɪ/ ⓘ; born 7 July 1981) is an Indian professional cricketer who plays as a right-handed batter and a wicket-keeper. Widely regarded as one of the most'),
 Document(metadata={'source': 'sample.txt'}, page_content='prolific wicket-keeper batsmen and captains, he represented the Indian cricket team and was the captain of the side in limited overs formats from 2007 to 2017 and in test cricket from 2008 to 2014.')]

# Text Embedding Models

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Vector Stores

In [14]:
db = Chroma.from_documents(texts, embeddings)

In [ ]:
db._collection.get(include=['embeddings'])

# Retrievers

In [16]:
retriever = db.as_retriever(search_kwargs={"k":3})

In [17]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x78ac132ff190>, search_kwargs={'k': 3})

# Question 1

In [18]:
question = input("enter your question?")

enter your question?who is dhoni's wife?


In [19]:
docs = retriever.get_relevant_documents(question)

<ipython-input-19-6ed304eeafec>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


In [20]:
docs

[Document(metadata={'source': 'sample.txt'}, page_content='Dehradun.[179][180] Dhoni and his wife have a daughter, Ziva Dhoni who was born on 6 February 2015.[181][182] He lives in his farmhouse outside Ranchi.[183] Dhoni is an automotive enthusiast and owns'),
 Document(metadata={'source': 'sample.txt'}, page_content='Dhoni is also known for his cool-headed demeanor on the field which has earned him the monicker "Captain cool".[178]\nPersonal life\n\nDhoni married Sakshi Singh Rawat on 4 July 2010 in'),
 Document(metadata={'source': 'sample.txt'}, page_content='wicket-keeper.[119] Dhoni is an unorthodox batsman who deviates from conventional coaching manuals to showcase his batting technique.[167] Dhoni displays a proclivity for hitting full-length')]

In [21]:
relevant_text = ''
for doc in docs:
  relevant_text += doc.page_content
print(relevant_text)

Dehradun.[179][180] Dhoni and his wife have a daughter, Ziva Dhoni who was born on 6 February 2015.[181][182] He lives in his farmhouse outside Ranchi.[183] Dhoni is an automotive enthusiast and ownsDhoni is also known for his cool-headed demeanor on the field which has earned him the monicker "Captain cool".[178]
Personal life

Dhoni married Sakshi Singh Rawat on 4 July 2010 inwicket-keeper.[119] Dhoni is an unorthodox batsman who deviates from conventional coaching manuals to showcase his batting technique.[167] Dhoni displays a proclivity for hitting full-length


In [22]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from google.colab import userdata

hf_token = userdata.get('HUGGINGFACE_HUB_API_KEY')

# Define the repository ID for the Gemma 2b model
repo_id = "google/gemma-2b"

# Set up a Hugging Face Endpoint for Gemma 2b model
llm = HuggingFaceEndpoint(
    repo_id=repo_id, max_length=1024, temperature=0.1, huggingfacehub_api_token=hf_token
)

<ipython-input-22-1e4f8a38b56e>:12: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceEndpoint(
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [23]:
template = """Answer the question in one line using the following information:

```{information}```.


*** Question ***

{question}

*** Answer ***"""

prompt = PromptTemplate.from_template(template=template)

In [25]:
prompt_formatted_str: str = prompt.format(question=question, information=relevant_text)
response = llm.invoke(prompt_formatted_str)
response.strip()

'sakshi singh rawat'